## Calculate ATO for TAZ centroid(s)

1. Run 1_setup_network.ipynb to create NetworkDataset_MM
2. Create OD Cost Matrix Layer
3. Add origin(s)
4. Add destinations
5. Solve
6. Join attributes to solved Lines layer
7. Weight HH and JOB by time decay
8. Sum

In [51]:
# Set to True to limit to ~30 TAZs
testing = True

input = "auto_baseline"
#input = "bus_baseline"
#input = r"scenario\auto_widen_12300"

In [52]:
if input == "auto_baseline":
    mode = "Driving"
    network = "baseline.gdb"
    
elif input == "bus_baseline":
    mode = "Transit"
    network = "baseline.gdb"

else:
    mode = network.split('_')[0]
    if mode not in ['Driving', 'Transit', 'Walking']:
        print("Please prepend network with 'auto', 'bus', or 'ped'")
        raise Exception
    network = os.path.join("scenario", input)

output_file = input

In [53]:

# input_network_dataset = os.path.join(network, r"NetworkDataset\NetworkDataset_ND")
# elif network == "mod":
#     # mod
#     target_gdb =  os.path.join(base_path, "mod.gdb")
# #arcpy.env.workspace = target_gdb
#     input_network_dataset = os.path.join(target_gdb, r"NetworkDataset\NetworkDataset_ND")
# else:
#     print("Please choose a valid input")

In [54]:
input = "mod"
network = r"scenario\auto_widen_12300.gdb"
output_file = input

In [55]:
import arcpy
import os

arcpy.CheckOutExtension("network")

from arcgis.features import SpatialDataFrame
import pandas as pd
# from arcgis.features import GeoAccessor, GeoSeriesAccessor

def survey_weight(t):
    if t <= 3:
        return 1
    elif (t > 3) & (t <= 20):
        return -0.0382 * t + 1.1293
    elif t > 20:
        return 1/(1 + math.exp(0.1092 * t - 1.5604))
    else:
        return 0

base_path = os.path.abspath(".")
network = os.path.join(base_path, network, r"NetworkDataset\NetworkDataset_ND")

arcpy.env.workspace = os.path.join(base_path, "ato.gdb")

In [56]:
nd_path = network
nd_layer_name = "wfrc_mm"
arcpy.nax.MakeNetworkDatasetLayer(nd_path, nd_layer_name)

<Result 'wfrc_mm'>

In [57]:
if testing:
    centroids = os.path.join(base_path, r"shp\taz_wfrc.gdb\taz_centroids_sample")
else:
    centroids = os.path.join(base_path, r"shp\taz_wfrc.gdb\taz_centroids_snapped")

# Create OD Cost Matrix Layers
# arcpy.na.MakeODCostMatrixAnalysisLayer(
#     network_data_source = network, 
#     layer_name = "OD Cost Matrix", 
#     travel_mode = mode, 
#     cutoff = 60.0,
#     line_shape = "NO_LINES"
# )

In [58]:
odcm = arcpy.nax.OriginDestinationCostMatrix(nd_layer_name)

In [59]:
odcm.travelMode = mode
odcm.defaultImpedanceCutoff = 60
odcm.lineShapeType = arcpy.nax.LineShapeType.NoLine

In [60]:
# Load inputs using field mappings, including the pre-calculated location fields
field_mappings_origins = odcm.fieldMappings(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, True)
# Map the "CO_TAZID" field in the input data to the "Name" property of the Origins class
field_mappings_origins["Name"].mappedFieldName = "CO_TAZID"
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, centroids, field_mappings_origins)
# Load destinations
field_mappings_destinations = odcm.fieldMappings(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, True)
field_mappings_destinations["Name"].mappedFieldName = "CO_TAZID"
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, centroids, field_mappings_destinations)

In [61]:
# Solve the analysis
result = odcm.solve()

# Export the results to a feature class
if result.solveSucceeded:
    result.export(arcpy.nax.OriginDestinationCostMatrixOutputDataType.Lines, "memory\output_lines")
else:
    print("Solve failed")
    print(result.solverMessages(arcpy.nax.MessageSeverity.All))

In [62]:
od = pd.DataFrame.spatial.from_featureclass(r"memory\output_lines")

In [63]:
od.head()

,ObjectID,DestinationRank,Total_Time,Total_Distance,OriginOID,OriginName,DestinationOID,DestinationName,SHAPE
0,1,1,0.000000,0.000000,1,351496,1,351496,None
1,2,2,1.814034,1.216938,1,351496,12,351498,None
2,3,3,2.382992,1.608512,1,351496,161,351499,None
3,4,4,2.417536,1.627814,1,351496,5,351497,None
4,5,5,2.511729,1.694244,1,351496,7,351495,None


In [64]:
assert od['Total_Time'].sum() > 100

In [65]:
taz = pd.DataFrame.spatial.from_featureclass(os.path.join(r"shp\taz_wfrc.gdb", "ATO"))

In [66]:
taz = taz[['CO_TAZID', 'HH_19', 'JOB_19', 'JOBAUTO_19', 'HHAUTO_19', 'JOBTRANSIT_19', 'HHTRANSIT_19']]

In [67]:
od['DestinationName'] = od['DestinationName'].astype(int)

In [68]:
df = pd.merge(od, taz, left_on="DestinationName", right_on="CO_TAZID", )

In [69]:
df.rename(columns={"OriginName": "Origin_TAZID",
                   "DestinationName": "Destination_TAZID"},
          inplace=True)

In [70]:
# Weight outputs
df['survey_weight'] = df['Total_Time'].apply(lambda x: survey_weight(x)).round(3)


df['weighted_jobs'] = df['survey_weight'] * df['JOB_19']
df['weighted_hh'] = df['survey_weight'] * df['HH_19']
df['weighted_jobs'] = round(df['weighted_jobs'])
df['weighted_hh'] = round(df['weighted_hh'])

df['ato'] = df['weighted_jobs'] + df['weighted_hh']

In [71]:
df.head()

,ObjectID,DestinationRank,Total_Time,Total_Distance,OriginOID,Origin_TAZID,DestinationOID,Destination_TAZID,SHAPE,CO_TAZID,HH_19,JOB_19,JOBAUTO_19,HHAUTO_19,JOBTRANSIT_19,HHTRANSIT_19,survey_weight,weighted_jobs,weighted_hh,ato
0,1,1,0.000000,0.000000,1,351496,1,351496,None,351496,279.0,324.7,231437,150510,3038,1525,1.000,325.0,279.0,604.0
1,187,4,2.581873,1.674189,2,351490,1,351496,None,351496,279.0,324.7,231437,150510,3038,1525,1.000,325.0,279.0,604.0
2,373,7,3.876465,2.542292,3,351488,1,351496,None,351496,279.0,324.7,231437,150510,3038,1525,0.981,319.0,274.0,593.0
3,557,8,3.772201,2.565888,4,351494,1,351496,None,351496,279.0,324.7,231437,150510,3038,1525,0.985,320.0,275.0,595.0
4,736,4,2.417536,1.627814,5,351497,1,351496,None,351496,279.0,324.7,231437,150510,3038,1525,1.000,325.0,279.0,604.0


In [72]:
# write to disk
df[['Origin_TAZID', 'Destination_TAZID', 'Total_Time', 'survey_weight',
         'weighted_jobs', 'weighted_hh', 'ato']].to_csv(output_file + '.csv')

In [73]:
taz_summary = df.groupby('Origin_TAZID').agg(
    jobs=pd.NamedAgg(column='weighted_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='weighted_hh', aggfunc=sum)
)
taz_summary['ato'] = taz_summary['jobs'] + taz_summary['hh']
taz_summary.to_csv(output_file + '_summary.csv')